## Initial Data Load

From https://dev.socrata.com/foundry/data.seattle.gov/4xy5-26gy

In [29]:
import pandas as pd
import pyodbc
import os
import snowflake.connector
from sodapy import Socrata
from sqlalchemy import types, create_engine
from dotenv import load_dotenv
from snowflake.connector.pandas_tools import write_pandas

load_dotenv()

True

Fetch data using `Socrata`

In [2]:
client = Socrata("data.seattle.gov", "lLbrAiAvALo3nTwseiBdQ4zWS")
results = client.get("65db-xm6k", limit = 200000)

Convert results to pandas DataFrame

In [3]:
results_df = pd.DataFrame.from_records(results)

In [4]:
results_df

,date,fremont_bridge,fremont_bridge_sb,fremont_bridge_nb
0,2021-04-30T23:00:00.000,36,12,24
1,2021-04-30T22:00:00.000,34,13,21
2,2021-04-30T21:00:00.000,34,14,20
3,2021-04-30T20:00:00.000,65,27,38
4,2021-04-30T19:00:00.000,139,64,75
...,...,...,...,...
141417,2012-10-03T02:00:00.000,2,1,1
141418,2012-10-03T01:00:00.000,10,4,6
141419,2012-10-03T01:00:00.000,10,4,6
141420,2012-10-03T00:00:00.000,13,4,9


In [5]:
results_df.shape

(141422, 4)

### ODBC

Connect to Snowflake ODBC DSN. I _think_ the way to do this is with SQLAlchemy.

References:
* https://docs.snowflake.com/en/user-guide/sqlalchemy.html#verifying-your-installation

In [19]:
engine = create_engine(
    'snowflake://{user}:{password}@{account}/{database}/{schema}'.format(
        user=os.environ.get('uid'),
        password=os.environ.get('pwd'),
        account=os.environ.get('account'),
        database='PYTHONATHON',
        schema="CONTENT"
    )
)

Verify that the connection can be made

In [20]:
try:
    connection = engine.connect()
    results = connection.execute('select current_schema()').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()

CONTENT


Try writing the pandas DataFrame into the DB

* First try with SQLAlchemy
* Snowflake doesn't support indexes?
* Maximum number of expressions allows is 16,384, hence the chunksize
* It doesn't look like SQLAlchemy infers column types. They need to be specifically passed via the `dtype` param
    * To figure out how to pass the proper types, I followed this SO post: https://stackoverflow.com/questions/55801668/the-type-of-field-is-not-a-sqlalchemy-type-with-pandas-to-sql-to-an-oracle-dat

In [31]:
conn = engine.connect()
col_types = dict(zip(results_df.columns.to_list(), (types.DateTime(), types.Integer(), types.Integer(), types.Integer(), types.Integer())))
results_df.to_sql(name = "bike_count",
                 con = conn,
                 if_exists = "replace",
                 index = False,
                 chunksize = 16000,
                 dtype = col_types)

The above chunk worked to create and write a new table **with the proper data types**. This was done using `SQLAlchemy` and the `pandas` `to_sql()` method. The below attempt using Snowflake's data connector didn't work due to the following:
* It appears the table has to be already created before the Snowflake connector can write to it. In the abolve `SQLAlchemy` example, `if_exists` can be used similar to R's `overwrite` argument.
* Once the table was created, there were issues getting data written into it. I stopped fighting with it and just came back to `SQLAlchemy`. The issues may have been caused by incorrectly specifying the datatype for the `date` column

In [ ]:
conn = snowflake.connector.connect(
    user = os.environ.get('uid'),
    password = os.environ.get('pwd'),
    account = os.environ.get('account'),
    database = 'PYTHONATHON',
    schema = "CONTENT"
)

cur = conn.cursor()

In [ ]:
cur.execute("SELECT current_database();")
cur.fetchone()[0]

In [34]:
sql = "DROP TABLE IF EXISTS BIKE_COUNT; CREATE TABLE BIKE_COUNT (date DATETIME, fremont_bridge NUMBER, fremont_bridge_sb NUMBER, fremont_bridge_nb NUMBER);" 
cur.execute(sql)

In [35]:
write_pandas(conn, results_df, "BIKE_COUNT")

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 85
invalid identifier '"date"'